# SSP Scenario Download API

* **Author:** Seongjun Lee
* **Date:** June 3, 2025
* **Description:** This Python script facilitates the download of climate element data for South Korea, categorized by Shared Socioeconomic Pathways (SSP) scenarios. It utilizes an API provided by the Korea Meteorological Administration (KMA).
* **Key Features:**
    * Retrieves climate data based on specified SSP scenarios.
    * Targets a 1km grid resolution for South Korea.
    * Allows selection of various climate elements (e.g., temperature, precipitation).
    * Constructs download URLs dynamically based on user-defined parameters such as scenario, climate element, period, and file format.
* **Note:** Users must replace the placeholder `authKey` with their valid API key provided by KMA. Data availability may vary depending on the specific combination of parameters selected.

## 0. Basic Settings

In [1]:
# Import libraries
import os
import requests
from tqdm import tqdm

# Save directory
save_dir = "/Users/lsj/NAS/SSP/South_Korea"

In [2]:
# Base API URL
base_url = "https://apihub-org.kma.go.kr/api/typ01/url/ssp_skorea_file_down.php"

# Auth Key
auth_key = "Or595fRNRNq-feX0TVTa7w" 

# SSP Scenario
ssp_scenarios = {
    "SSP1_2.6": "SSP126",
    "SSP2_4.5": "SSP245",
    "SSP3_7.0": "SSP370",
    "SSP5_8.5": "SSP585",
}

# Basic Climate Elements
basic_elements = {
    "avg_temp": "TA",                   # 평균기온 (Average Temperature)
    "max_temp": "TAMAX",                # 최고기온 (Maximum Temperature)
    "min_temp": "TAMIN",                # 최저기온 (Minimum Temperature)
    "precip": "RN",                     # 강수량 (Precipitation) 
    "rh": "RHM",                        # 상대습도 (Relative Humidity) 
    "avg_wind_speed": "WS",             # 평균풍속 (Average Wind Speed) 
    "sol_rad": "SI"                     # 일사량 (Solar Radiation)
    # "sst": "SST",                     # 해수면온도 (Sea Surface Temperature)              # No daily data
    # "max_wind_speed": "WSMAX",        # 최대풍속 (Maximum Wind Speed)                     # No daily data
    # "salinity": "SALNT",              # 표층염분 (Surface Salinity)                       # No daily data
    # "sea_ice_conc": "SICONCA",        # 해빙면적 (Sea Ice Concentration)                  # No daily data
    # "sea_level_height": "SLR",        # 해수면고도 (Sea Level Rise / Sea Surface Height)  # No daily data 
    # "slp": "PS",                      # 해면기압 (Sea Level Pressure)                     # No daily data
    # "surface_pressure": "PA",         # 현지기압 (Station Pressure / Surface Pressure)    # No daily data
    # "snow_depth": "SD"                # 적설량 (Snow Depth)                               # No daily data
}

# URL Other Parameters
model = "5ENSMN"       
grid = "gridraw"          # 1km grid
time_rsltn = "daily"      # Daily or yearly
frmat = "nc"             # NetCDF file format

# 10-year period
periods = []
for year_start in range(2021, 2100, 10):
    year_end = year_start + 9
    periods.append({"st_year": str(year_start), "ed_year": str(year_end)})

## 1. Download Data using API

In [ ]:
# Create save directory (if not exists)
os.makedirs(save_dir, exist_ok=True)

# Calculate total number of tasks
total_tasks = len(periods) * len(ssp_scenarios) * len(basic_elements)

# Progress bar
with tqdm(total=total_tasks, desc="Download Progress") as pbar:
    for period in periods:
        st_year_val = period["st_year"]
        ed_year_val = period["ed_year"]
        
        for scenario_label, rpt_val in ssp_scenarios.items():
            for element_label, elem_val in basic_elements.items():
                params = {
                    "rpt": rpt_val,
                    "model": model,
                    "elem": elem_val,
                    "grid": grid,
                    "time_rsltn": time_rsltn, 
                    "st_year": st_year_val,   
                    "ed_year": ed_year_val,   
                    "frmat": frmat,
                    "authKey": auth_key,
                }
                
                query_string = "&".join([f"{key}={value}" for key, value in params.items()])
                api_url = f"{base_url}?{query_string}"

                file_name = f"{rpt_val}_{element_label}_daily_{st_year_val}_{ed_year_val}.tar.gz"
                file_path = os.path.join(save_dir, file_name)
                
                try:
                    response = requests.get(api_url, stream=True)
                    response.raise_for_status()
                    
                    # Check file size
                    total_size = int(response.headers.get('content-length', 0))
                    
                    # Save file and show progress
                    with open(file_path, 'wb') as f:
                        if total_size == 0:  # If file size is unknown
                            f.write(response.content)
                        else:
                            downloaded = 0
                            for data in response.iter_content(chunk_size=4096):
                                downloaded += len(data)
                                f.write(data)
                                # Show progress bar (MB unit)
                                pbar.set_postfix({
                                    'file': file_name,
                                    'size': f'{round(downloaded/(1024*1024), 2)}/{round(total_size/(1024*1024), 2)} MB'
                                })
                    
                except requests.exceptions.RequestException as e:
                    tqdm.write(f"Download failed: {file_name} - {str(e)}")
                
                pbar.update(1)

Download Progress:  12%|█▏        | 26/224 [13:08<1:29:38, 27.17s/it, file=SSP585_avg_wind_speed_daily_2021_2030.tar.gz, size=72.51/204.01 MB] 